In [51]:
%%bash
ifconfig

eth0      Link encap:Ethernet  HWaddr 00:0d:3a:94:51:73  
          inet addr:10.0.0.18  Bcast:10.0.15.255  Mask:255.255.240.0
          inet6 addr: fe80::20d:3aff:fe94:5173/64 Scope:Link
          UP BROADCAST RUNNING MULTICAST  MTU:1500  Metric:1
          RX packets:37331739 errors:0 dropped:1226 overruns:0 frame:0
          TX packets:24550303 errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:1000 
          RX bytes:24660725468 (24.6 GB)  TX bytes:15173808346 (15.1 GB)

lo        Link encap:Local Loopback  
          inet addr:127.0.0.1  Mask:255.0.0.0
          inet6 addr: ::1/128 Scope:Host
          UP LOOPBACK RUNNING  MTU:65536  Metric:1
          RX packets:26665321 errors:0 dropped:0 overruns:0 frame:0
          TX packets:26665321 errors:0 dropped:0 overruns:0 carrier:0
          collisions:0 txqueuelen:1 
          RX bytes:14476053440 (14.4 GB)  TX bytes:14476053440 (14.4 GB)



In [1]:
%%configure -f
{
    "conf":{
        "spark.submit.pyFiles":"wasb://h2ofiles@****/pySparkling.egg",
        "spark.master":"local[*]"
    },
    "driverMemory":"8G",
    "executorMemory":"8G",
    "numExecutors":1
}

ERROR:root:Cell magic `%%configure` not found.


In [53]:
sc

In [54]:
import pysparkling
import os
os.environ["PYTHON_EGG_CACHE"] = "~/"

In [55]:
import numpy as np
import pandas as pd
import pyspark
import os
import urllib
import sys

from pyspark.sql.functions import *
from pyspark.ml.classification import *
from pyspark.ml.evaluation import *
from pyspark.ml.feature import *


In [56]:
from pyspark.mllib.classification import *

In [57]:
# start Spark session
spark = pyspark.sql.SparkSession.builder.appName('Iris').getOrCreate()

# print runtime versions
print ('****************')
print ('Python version: {}'.format(sys.version))
print ('Spark version: {}'.format(spark.version))
print ('****************')





****************
Python version: 2.7.12 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:42:40) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]
Spark version: 2.0.2.2.5.4.2-7
****************

In [58]:
# load iris.csv into Spark dataframe
data = sqlContext.read.csv('wasbs://h2ofiles@****/iris.csv', header=None,inferSchema=True, nanValue="")
                         
']))





In [59]:
data.describe()
names=['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
oldnames = data.columns
from pyspark.sql.functions import col





In [60]:
data.show(10)

+---+---+---+---+-----------+
|_c0|_c1|_c2|_c3|        _c4|
+---+---+---+---+-----------+
|5.1|3.5|1.4|0.2|Iris-setosa|
|4.9|3.0|1.4|0.2|Iris-setosa|
|4.7|3.2|1.3|0.2|Iris-setosa|
|4.6|3.1|1.5|0.2|Iris-setosa|
|5.0|3.6|1.4|0.2|Iris-setosa|
|5.4|3.9|1.7|0.4|Iris-setosa|
|4.6|3.4|1.4|0.3|Iris-setosa|
|5.0|3.4|1.5|0.2|Iris-setosa|
|4.4|2.9|1.4|0.2|Iris-setosa|
|4.9|3.1|1.5|0.1|Iris-setosa|
+---+---+---+---+-----------+
only showing top 10 rows

In [61]:

newdf = data.select([col(oldnames[index]).alias(names[index]) for index in xrange(len(oldnames))])

In [62]:
newdf.describe().show()

+-------+------------------+-------------------+------------------+------------------+
|summary|      sepal-length|        sepal-width|      petal-length|       petal-width|
+-------+------------------+-------------------+------------------+------------------+
|  count|               150|                150|               150|               150|
|   mean| 5.843333333333335| 3.0540000000000007|3.7586666666666693|1.1986666666666672|
| stddev|0.8280661279778637|0.43359431136217375| 1.764420419952262|0.7631607417008414|
|    min|               4.3|                2.0|               1.0|               0.1|
|    max|               7.9|                4.4|               6.9|               2.5|
+-------+------------------+-------------------+------------------+------------------+

In [63]:
print "First 20 rows of Iris dataset:"
newdf.describe('class').show()

First 20 rows of Iris dataset:
+-------+--------------+
|summary|         class|
+-------+--------------+
|  count|           150|
|   mean|          null|
| stddev|          null|
|    min|   Iris-setosa|
|    max|Iris-virginica|
+-------+--------------+

In [64]:
newdf.select('class').distinct().show()

+---------------+
|          class|
+---------------+
| Iris-virginica|
|    Iris-setosa|
|Iris-versicolor|
+---------------+

In [65]:
type(newdf)


<class 'pyspark.sql.dataframe.DataFrame'>

In [66]:


# vectorize all numerical columns into a single feature column
feature_cols = newdf.columns[:-1]
assembler = pyspark.ml.feature.VectorAssembler(inputCols=feature_cols, outputCol='features')
assembler_data = assembler.transform(newdf)

assembler_data.show()



+------------+-----------+------------+-----------+-----------+-----------------+
|sepal-length|sepal-width|petal-length|petal-width|      class|         features|
+------------+-----------+------------+-----------+-----------+-----------------+
|         5.1|        3.5|         1.4|        0.2|Iris-setosa|[5.1,3.5,1.4,0.2]|
|         4.9|        3.0|         1.4|        0.2|Iris-setosa|[4.9,3.0,1.4,0.2]|
|         4.7|        3.2|         1.3|        0.2|Iris-setosa|[4.7,3.2,1.3,0.2]|
|         4.6|        3.1|         1.5|        0.2|Iris-setosa|[4.6,3.1,1.5,0.2]|
|         5.0|        3.6|         1.4|        0.2|Iris-setosa|[5.0,3.6,1.4,0.2]|
|         5.4|        3.9|         1.7|        0.4|Iris-setosa|[5.4,3.9,1.7,0.4]|
|         4.6|        3.4|         1.4|        0.3|Iris-setosa|[4.6,3.4,1.4,0.3]|
|         5.0|        3.4|         1.5|        0.2|Iris-setosa|[5.0,3.4,1.5,0.2]|
|         4.4|        2.9|         1.4|        0.2|Iris-setosa|[4.4,2.9,1.4,0.2]|
|         4.9|  

In [67]:
# convert text labels into indices
label_data = assembler_data.select(['features'] + [ 'class'])
label_indexer = pyspark.ml.feature.StringIndexer(inputCol='class', outputCol='label').fit(assembler_data)
labelled_data = label_indexer.transform(label_data)

In [68]:
labelled_data.show()

+-----------------+-----------+-----+
|         features|      class|label|
+-----------------+-----------+-----+
|[5.1,3.5,1.4,0.2]|Iris-setosa|  0.0|
|[4.9,3.0,1.4,0.2]|Iris-setosa|  0.0|
|[4.7,3.2,1.3,0.2]|Iris-setosa|  0.0|
|[4.6,3.1,1.5,0.2]|Iris-setosa|  0.0|
|[5.0,3.6,1.4,0.2]|Iris-setosa|  0.0|
|[5.4,3.9,1.7,0.4]|Iris-setosa|  0.0|
|[4.6,3.4,1.4,0.3]|Iris-setosa|  0.0|
|[5.0,3.4,1.5,0.2]|Iris-setosa|  0.0|
|[4.4,2.9,1.4,0.2]|Iris-setosa|  0.0|
|[4.9,3.1,1.5,0.1]|Iris-setosa|  0.0|
|[5.4,3.7,1.5,0.2]|Iris-setosa|  0.0|
|[4.8,3.4,1.6,0.2]|Iris-setosa|  0.0|
|[4.8,3.0,1.4,0.1]|Iris-setosa|  0.0|
|[4.3,3.0,1.1,0.1]|Iris-setosa|  0.0|
|[5.8,4.0,1.2,0.2]|Iris-setosa|  0.0|
|[5.7,4.4,1.5,0.4]|Iris-setosa|  0.0|
|[5.4,3.9,1.3,0.4]|Iris-setosa|  0.0|
|[5.1,3.5,1.4,0.3]|Iris-setosa|  0.0|
|[5.7,3.8,1.7,0.3]|Iris-setosa|  0.0|
|[5.1,3.8,1.5,0.3]|Iris-setosa|  0.0|
+-----------------+-----------+-----+
only showing top 20 rows

In [69]:
# only select the features and label column
processed_data = labelled_data.select(['features', 'label'])
print("Reading for machine learning")
processed_data.show(10)

Reading for machine learning
+-----------------+-----+
|         features|label|
+-----------------+-----+
|[5.1,3.5,1.4,0.2]|  0.0|
|[4.9,3.0,1.4,0.2]|  0.0|
|[4.7,3.2,1.3,0.2]|  0.0|
|[4.6,3.1,1.5,0.2]|  0.0|
|[5.0,3.6,1.4,0.2]|  0.0|
|[5.4,3.9,1.7,0.4]|  0.0|
|[4.6,3.4,1.4,0.3]|  0.0|
|[5.0,3.4,1.5,0.2]|  0.0|
|[4.4,2.9,1.4,0.2]|  0.0|
|[4.9,3.1,1.5,0.1]|  0.0|
+-----------------+-----+
only showing top 10 rows

In [70]:
label_indexer

StringIndexer_43929aef1d4e117361db

In [71]:
# change regularization rate and you will likely get a different accuracy.
reg = 0.01
# load regularization rate from argument if present
if len(sys.argv) > 1:
    reg = float(sys.argv[1])

# log regularization rate
#run_logger.log("Regularization Rate", reg)

# use Logistic Regression to train on the training set
train, test = processed_data.randomSplit([0.70, 0.30])



In [72]:
train.show()

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[4.3,3.0,1.1,0.1]|  0.0|
|[4.4,3.0,1.3,0.2]|  0.0|
|[4.4,3.2,1.3,0.2]|  0.0|
|[4.5,2.3,1.3,0.3]|  0.0|
|[4.6,3.1,1.5,0.2]|  0.0|
|[4.6,3.2,1.4,0.2]|  0.0|
|[4.6,3.6,1.0,0.2]|  0.0|
|[4.7,3.2,1.3,0.2]|  0.0|
|[4.7,3.2,1.6,0.2]|  0.0|
|[4.8,3.0,1.4,0.1]|  0.0|
|[4.8,3.4,1.6,0.2]|  0.0|
|[4.8,3.4,1.9,0.2]|  0.0|
|[4.9,2.4,3.3,1.0]|  1.0|
|[4.9,2.5,4.5,1.7]|  2.0|
|[4.9,3.0,1.4,0.2]|  0.0|
|[4.9,3.1,1.5,0.1]|  0.0|
|[4.9,3.1,1.5,0.1]|  0.0|
|[4.9,3.1,1.5,0.1]|  0.0|
|[5.0,2.0,3.5,1.0]|  1.0|
|[5.0,2.3,3.3,1.0]|  1.0|
+-----------------+-----+
only showing top 20 rows

In [73]:
from pyspark.mllib import linalg as mllib_linalg
from pyspark.ml import linalg as ml_linalg

def as_old(v):
    if isinstance(v, ml_linalg.SparseVector):
        return mllib_linalg.SparseVector(v.size, v.indices, v.values)
    if isinstance(v, ml_linalg.DenseVector):
        return mllib_linalg.DenseVector(v.values)
    raise ValueError("Unsupported type {0}".format(type(v)))

In [74]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import col

train_labelpoint = (train.select(col("label").alias("label"), col("features"))
  .rdd
  .map(lambda row: LabeledPoint(row.label, as_old(row.features))))

In [75]:
train_labelpoint

PythonRDD[58] at RDD at PythonRDD.scala:48

In [76]:
lrm = LogisticRegressionWithLBFGS.train(train_labelpoint, iterations=10, numClasses=3)

In [77]:
lrm.predict([4.3,3.0,1.1,0.1])

0

In [78]:
result = [lrm.predict(x) for x in test.select('features').rdd.map(lambda r: as_old(r[0])).collect()]

In [79]:
result

[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 2, 1, 1, 2, 1, 1, 0, 1, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 2]

In [80]:
def simple(x):
    return x
simpleUdf = udf(simple, DoubleType())

name 'DoubleType' is not defined
Traceback (most recent call last):
NameError: name 'DoubleType' is not defined



In [81]:
lrm.predict([4.3,3.0,1.1,0.1])

0

In [82]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
def predict(x):
    a = as_old(x)
    return lrm.predict(a)
predictUdf = udf(predict, IntegerType())

In [83]:
test_predicted = test.withColumn('prediction', predictUdf(test.features)) 
test_predicted.count()
test_predicted.show()

+-----------------+-----+----------+
|         features|label|prediction|
+-----------------+-----+----------+
|[4.4,2.9,1.4,0.2]|  0.0|         0|
|[4.6,3.4,1.4,0.3]|  0.0|         0|
|[4.8,3.0,1.4,0.3]|  0.0|         0|
|[4.8,3.1,1.6,0.2]|  0.0|         0|
|[5.0,3.4,1.6,0.4]|  0.0|         0|
|[5.1,2.5,3.0,1.1]|  1.0|         1|
|[5.2,3.4,1.4,0.2]|  0.0|         0|
|[5.2,4.1,1.5,0.1]|  0.0|         0|
|[5.3,3.7,1.5,0.2]|  0.0|         0|
|[5.4,3.0,4.5,1.5]|  1.0|         1|
|[5.4,3.7,1.5,0.2]|  0.0|         0|
|[5.4,3.9,1.7,0.4]|  0.0|         0|
|[5.5,2.5,4.0,1.3]|  1.0|         1|
|[5.5,4.2,1.4,0.2]|  0.0|         0|
|[5.6,2.5,3.9,1.1]|  1.0|         1|
|[5.6,2.8,4.9,2.0]|  2.0|         2|
|[5.6,2.9,3.6,1.3]|  1.0|         1|
|[5.6,3.0,4.1,1.3]|  1.0|         1|
|[5.7,2.5,5.0,2.0]|  2.0|         2|
|[5.7,2.6,3.5,1.0]|  1.0|         1|
+-----------------+-----+----------+
only showing top 20 rows

In [84]:
from pyspark.ml.linalg import Vector as MLVector, Vectors as MLVectors
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors
from pyspark.mllib.regression import  LabeledPoint

In [85]:
isinstance(test.features[0], MLLibVector)

False

In [86]:
isinstance(test.features[0], MLVector)

False

In [87]:
from pyspark.ml import linalg as ml_linalg

def as_mllib(v):
    if isinstance(v, ml_linalg.SparseVector):
        return MLLibVectors.sparse(v.size, v.indices, v.values)
    elif isinstance(v, ml_linalg.DenseVector):
        return MLLibVectors.dense(v.toArray())
    else:
        raise TypeError("Unsupported type: {0}".format(type(v)))

In [88]:
def predictionAndLabels(a,b):
    return (a,b)
result = predictionAndLabels(1,2)
predictionAndLabelsUdf = udf(predictionAndLabels, IntegerType())
print result
type(result)

(1, 2)
<type 'tuple'>

In [89]:
testperf =  test_predicted.withColumn('predictionAndLabels', predictionAndLabelsUdf(test_predicted.prediction, test_predicted.label))

In [90]:
predictionAndLabelsList = test_predicted.select('prediction','label').rdd.map(lambda r: (r[0]*1.0, r[1])).collect()

In [91]:
predictionAndLabelsList

[(0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 1.0), (0.0, 0.0), (0.0, 0.0), (0.0, 0.0), (1.0, 1.0), (0.0, 0.0), (0.0, 0.0), (1.0, 1.0), (0.0, 0.0), (1.0, 1.0), (2.0, 2.0), (1.0, 1.0), (1.0, 1.0), (2.0, 2.0), (1.0, 1.0), (1.0, 1.0), (0.0, 0.0), (1.0, 1.0), (2.0, 2.0), (1.0, 1.0), (2.0, 2.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (1.0, 1.0), (2.0, 2.0), (2.0, 2.0), (2.0, 2.0), (2.0, 2.0), (1.0, 1.0), (2.0, 2.0), (2.0, 2.0), (2.0, 2.0), (1.0, 1.0), (1.0, 1.0), (2.0, 2.0), (2.0, 2.0), (2.0, 2.0)]

In [92]:
from pyspark.mllib.evaluation import MulticlassMetrics


In [93]:
result = sc.parallelize(list(predictionAndLabelsList))

In [94]:

metrics = MulticlassMetrics(result)

In [95]:
metrics.confusionMatrix().toArray()

array([[ 12.,   0.,   0.],
       [  0.,  17.,   0.],
       [  0.,   0.,  14.]])

In [96]:
print "falsePositiveRate is " + str(metrics.falsePositiveRate(0.0))
print "recall is %f" %( metrics.recall(2.0))
print "precision is %f " %(metrics.precision(1.0))
print "fMeasure is %f" %(metrics.fMeasure(0.0,2.0))
print "accuracy is %f" %(metrics.accuracy)


falsePositiveRate is 0.0
recall is 1.000000
precision is 1.000000 
fMeasure is 1.000000
accuracy is 1.000000